# Create NSF Awards from NSF Award Search

Creates NSF awards from NSF Award Search data (1960-present). 

**Prerequisites:**
- Run `scripts/local/nsf_awards_to_s3.py` to download and upload the data first.

**Data source:** https://www.nsf.gov/awardsearch/download-awards  
**S3 location:** `s3a://openalex-ingest/awards/nsf/nsf_awards_combined.parquet`

**NSF funder in OpenAlex:**
- funder_id: 4320306076
- display_name: "National Science Foundation"
- ror_id: "https://ror.org/021nxhr62"
- doi: "10.13039/100000001"

## Step 1: Create Staging Table

In [ ]:
%sql
-- Create the staging table from S3 parquet
CREATE OR REPLACE TABLE openalex.awards.nsf_grants_raw
USING delta
AS
SELECT
    *,
    current_timestamp() as ingested_at
FROM parquet.`s3a://openalex-ingest/awards/nsf/nsf_awards_combined.parquet`;

In [ ]:
%sql
-- Check row count
SELECT COUNT(*) as total_grants FROM openalex.awards.nsf_grants_raw;

In [ ]:
%sql
-- Preview a few rows to inspect the schema
SELECT * FROM openalex.awards.nsf_grants_raw LIMIT 5;

## Step 2: Find NSF Funder ID

In [ ]:
%sql
-- Find NSF funder in OpenAlex
SELECT funder_id, display_name, ror_id, doi
FROM openalex.common.funder
WHERE LOWER(display_name) LIKE '%national science foundation%'
   OR ror_id = 'https://ror.org/021nxhr62'
   OR doi = '10.13039/100000001';

## Step 3: Create NSF Awards Table

In [ ]:
%sql
CREATE OR REPLACE TABLE openalex.awards.nsf_awards
USING delta
AS
WITH nsf_funder AS (
    -- NSF funder from OpenAlex (update funder_id after running Step 2)
    SELECT
        funder_id,
        display_name,
        ror_id,
        doi
    FROM openalex.common.funder
    WHERE ror_id = 'https://ror.org/021nxhr62'  -- National Science Foundation
),

awards_transformed AS (
    SELECT
        -- Generate unique ID using xxhash64 of funder + award_id
        abs(xxhash64(CONCAT(f.funder_id, ':', LOWER(g.awd_id)))) % 9000000000 as id,

        -- Display name = award title
        g.awd_titl_txt as display_name,

        -- Description = abstract
        g.awd_abstract_narration as description,

        -- Funder info
        f.funder_id,
        LOWER(g.awd_id) as funder_award_id,

        -- Funding amount
        CAST(g.awd_amount AS DOUBLE) as amount,
        'USD' as currency,

        -- Funder struct
        struct(
            CONCAT('https://openalex.org/F', f.funder_id) as id,
            f.display_name,
            f.ror_id,
            f.doi
        ) as funder,

        -- Funding type based on transaction type and instrument
        CASE
            WHEN LOWER(g.awd_istr_txt) LIKE '%fellowship%' THEN 'fellowship'
            WHEN LOWER(g.awd_istr_txt) LIKE '%cooperative%' THEN 'cooperative_agreement'
            WHEN LOWER(g.awd_istr_txt) LIKE '%contract%' THEN 'contract'
            WHEN LOWER(g.awd_istr_txt) LIKE '%continuing%' THEN 'grant'
            WHEN LOWER(g.awd_istr_txt) LIKE '%standard%' THEN 'grant'
            ELSE 'grant'
        END as funding_type,

        -- Funder scheme = award instrument type
        g.awd_istr_txt as funder_scheme,

        -- Provenance
        'nsf_award_search' as provenance,

        -- Dates (NSF dates are in yyyy-MM-dd format)
        TRY_TO_DATE(g.awd_eff_date, 'yyyy-MM-dd') as start_date,
        TRY_TO_DATE(g.awd_exp_date, 'yyyy-MM-dd') as end_date,
        YEAR(TRY_TO_DATE(g.awd_eff_date, 'yyyy-MM-dd')) as start_year,
        YEAR(TRY_TO_DATE(g.awd_exp_date, 'yyyy-MM-dd')) as end_year,

        -- Lead investigator
        CASE
            WHEN g.pi_last_name IS NOT NULL THEN
                struct(
                    g.pi_first_name as given_name,
                    g.pi_last_name as family_name,
                    CAST(NULL AS STRING) as orcid,
                    CAST(NULL AS DATE) as role_start,
                    struct(
                        g.inst_name as name,
                        g.inst_country_name as country,
                        CAST(NULL AS ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>) as ids
                    ) as affiliation
                )
            ELSE NULL
        END as lead_investigator,

        -- Co-lead and other investigators (NULL for now)
        CAST(NULL AS STRUCT<
            given_name:STRING,
            family_name:STRING,
            orcid:STRING,
            role_start:DATE,
            affiliation:STRUCT<name:STRING, country:STRING, ids:ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>>
        >) as co_lead_investigator,

        CAST(NULL AS ARRAY<STRUCT<
            given_name:STRING,
            family_name:STRING,
            orcid:STRING,
            role_start:DATE,
            affiliation:STRUCT<name:STRING, country:STRING, ids:ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>>
        >>) as investigators,

        -- Landing page URL (NSF Award Search page)
        CONCAT('https://www.nsf.gov/awardsearch/showAward?AWD_ID=', g.awd_id) as landing_page_url,

        -- No DOI for NSF grants typically
        CAST(NULL AS STRING) as doi,

        -- Works API URL
        concat('https://api.openalex.org/works?filter=awards.id:G', abs(xxhash64(CONCAT(f.funder_id, ':', LOWER(g.awd_id)))) % 9000000000) as works_api_url,

        -- Timestamps
        current_timestamp() as created_date,
        current_timestamp() as updated_date

    FROM openalex.awards.nsf_grants_raw g
    CROSS JOIN nsf_funder f
    WHERE g.awd_id IS NOT NULL
      AND TRIM(g.awd_id) != ''
)

SELECT * FROM awards_transformed;

In [ ]:
%sql
-- Remove previous data for this source before inserting fresh data
DELETE FROM openalex.awards.openalex_awards_raw
WHERE provenance = 'nsf_award_search' AND priority = 3;

-- Insert into openalex_awards_raw with priority
INSERT INTO openalex.awards.openalex_awards_raw
SELECT
    id,
    display_name,
    description,
    funder_id,
    funder_award_id,
    amount,
    currency,
    funder,
    funding_type,
    funder_scheme,
    provenance,
    start_date,
    end_date,
    start_year,
    end_year,
    lead_investigator,
    co_lead_investigator,
    investigators,
    landing_page_url,
    doi,
    works_api_url,
    created_date,
    updated_date,
    3 as priority
FROM openalex.awards.nsf_awards;

## Verification Queries

In [ ]:
%sql
-- Check row count
SELECT COUNT(*) as total_nsf_awards FROM openalex.awards.nsf_awards;

In [ ]:
%sql
-- Sample the data
SELECT * FROM openalex.awards.nsf_awards LIMIT 10;

In [ ]:
%sql
-- Check funding type distribution
SELECT funding_type, COUNT(*) as cnt
FROM openalex.awards.nsf_awards
GROUP BY funding_type
ORDER BY cnt DESC;

In [ ]:
%sql
-- Check funder_scheme (award instrument type) distribution
SELECT funder_scheme, COUNT(*) as cnt
FROM openalex.awards.nsf_awards
GROUP BY funder_scheme
ORDER BY cnt DESC
LIMIT 20;

In [ ]:
%sql
-- Check date parsing success rate
SELECT
    COUNT(*) as total,
    COUNT(start_date) as has_start_date,
    COUNT(end_date) as has_end_date,
    ROUND(COUNT(start_date) * 100.0 / COUNT(*), 1) as pct_with_start_date
FROM openalex.awards.nsf_awards;

In [ ]:
%sql
-- Check awards by year
SELECT start_year, COUNT(*) as cnt
FROM openalex.awards.nsf_awards
WHERE start_year IS NOT NULL
GROUP BY start_year
ORDER BY start_year DESC
LIMIT 20;

In [ ]:
%sql
-- Check top institutions by award count
SELECT lead_investigator.affiliation.name as institution, COUNT(*) as cnt
FROM openalex.awards.nsf_awards
WHERE lead_investigator.affiliation.name IS NOT NULL
GROUP BY lead_investigator.affiliation.name
ORDER BY cnt DESC
LIMIT 20;

In [ ]:
%sql
-- Summary statistics
SELECT
    COUNT(*) as total_awards,
    COUNT(DISTINCT funder_award_id) as unique_award_ids,
    MIN(start_year) as earliest_year,
    MAX(start_year) as latest_year,
    SUM(amount) as total_funding,
    AVG(amount) as avg_award_amount,
    PERCENTILE(amount, 0.5) as median_award_amount
FROM openalex.awards.nsf_awards;